In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import pyspark.sql.functions as F
from pyspark.ml.feature import (
   Tokenizer, 
    StopWordsRemover, 
    HashingTF, 
    IDF, 
    StringIndexer
)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes

In [2]:
# get or create Spark session

app_name = "job-john-snow"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [3]:
file_uri = "/mnt/mnt_project3/job.csv"

df = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(file_uri)
df.printSchema()

root
-- job_id: integer (nullable = true)
-- title: string (nullable = true)
-- location: string (nullable = true)
-- company_profile: string (nullable = true)
-- description: string (nullable = true)
-- requirements: string (nullable = true)
-- benefits: string (nullable = true)
-- telecommuting: string (nullable = true)
-- has_company_logo: string (nullable = true)
-- has_questions: string (nullable = true)
-- employment_type: string (nullable = true)
-- required_experience: string (nullable = true)
-- required_education: string (nullable = true)
-- industry: string (nullable = true)
-- function: string (nullable = true)
-- fraudulent: string (nullable = true)

In [4]:
# Create a length column to be used as a future feature 
df=df.filter(df.description.isNotNull())
df=df.filter(df.fraudulent.isNotNull())
data_df = df.withColumn('length', F.length(df['description']))
data=data_df.select('description','fraudulent','length')
data.show()

+--------------------+----------+------+
 description|fraudulent|length|
+--------------------+----------+------+
Food52, a fast-gr...| 0| 905|
Organised - Focus...| 0| 2077|
Our client, locat...| 0| 355|
THE COMPANY: ESRI...| 0| 2600|
JOB TITLE: Itemiz...| 0| 1520|
Job OverviewApex ...| 0| 3418|
Your Responsibili...| 0| 433|
Who is Airenvy?He...| 0| 2488|
Implementation/Co...| 0| 75|
The Customer Serv...| 0| 1219|
Position : #URL_8...| 0| 2878|
TransferWise is t...| 0| 1159|
The Applications ...| 0| 1721|
Event Industry In...| 0| 793|
Are you intereste...| 0| 986|
About Vault Drago...| Sales| 2995|
We are looking fo...| 0| 314|
Government fundin...| 0| 390|
Kettle is hiring ...| 0| 1561|
Experienced Proce...| 0| 1391|
+--------------------+----------+------+
only showing top 20 rows

In [5]:
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()
data.count()

+-----------+----------+------+
description|fraudulent|length|
+-----------+----------+------+
 0| 0| 0|
+-----------+----------+------+

Out[152]: 17704

Out[134]: 17879

In [7]:
data_df2=data.filter(data.fraudulent<2)
data_df2.show()

+--------------------+----------+------+
 description|fraudulent|length|
+--------------------+----------+------+
Food52, a fast-gr...| 0| 905|
Organised - Focus...| 0| 2077|
Our client, locat...| 0| 355|
THE COMPANY: ESRI...| 0| 2600|
JOB TITLE: Itemiz...| 0| 1520|
Job OverviewApex ...| 0| 3418|
Your Responsibili...| 0| 433|
Who is Airenvy?He...| 0| 2488|
Implementation/Co...| 0| 75|
The Customer Serv...| 0| 1219|
Position : #URL_8...| 0| 2878|
TransferWise is t...| 0| 1159|
The Applications ...| 0| 1721|
Event Industry In...| 0| 793|
Are you intereste...| 0| 986|
We are looking fo...| 0| 314|
Government fundin...| 0| 390|
Kettle is hiring ...| 0| 1561|
Experienced Proce...| 0| 1391|
IntelliBright is ...| 0| 1273|
+--------------------+----------+------+
only showing top 20 rows

In [8]:
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='fraudulent',outputCol='label')
tokenizer = Tokenizer(inputCol="description", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [9]:

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [10]:
data_prep_pipeline = Pipeline(stages=[
  pos_neg_to_num,
    tokenizer, 
    stopremove, 
    hashingTF, 
    idf, 
    clean_up
])

In [11]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df2)
cleaned = cleaner.transform(data_df2)
cleaned.show(10)

+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 description|fraudulent|length|label| token_text| stop_tokens| hash_token| idf_token| features|
+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
Food52, a fast-gr...| 0| 905| 0.0|[food52,, a, fast...|[food52,, fast-gr...|(262144,[5213,901...|(262144,[5213,901...|(262145,[5213,901...|
Organised - Focus...| 0| 2077| 0.0|[organised, -, fo...|[organised, -, fo...|(262144,[619,1665...|(262144,[619,1665...|(262145,[619,1665...|
Our client, locat...| 0| 355| 0.0|[our, client,, lo...|[client,, located...|(262144,[1156,384...|(262144,[1156,384...|(262145,[1156,384...|
THE COMPANY: ESRI...| 0| 2600| 0.0|[the, company:, e...|[company:, esri, ...|(262144,[2265,271...|(262144,[2265,271...|(262145,[2265,271...|
JOB TITLE: Itemiz...| 0| 1520| 0.0|[job, title:, ite...|[job, title:, ite...|(262144,[1232,595...|(262144,[1232,595...|(262145,[1232,595...|
Job OverviewApex ...| 0| 3418| 0.0|[job, overviewape...|[job, overviewape...|(262144,[619,1156...|(262144,[619,1156...|(262145,[619,1156...|
Your Responsibili...| 0| 433| 0.0|[your, responsibi...|[responsibilities...|(262144,[6962,157...|(262144,[6962,157...|(262145,[6962,157...|
Who is Airenvy?He...| 0| 2488| 0.0|[who, is, airenvy...|[airenvy?hey, the...|(262144,[1865,267...|(262144,[1865,267...|(262145,[1865,267...|
Implementation/Co...| 0| 75| 0.0|[implementation/c...|[implementation/c...|(262144,[24113,51...|(262144,[24113,51...|(262145,[24113,51...|
The Customer Serv...| 0| 1219| 0.0|[the, customer, s...|[customer, servic...|(262144,[15,1156,...|(262144,[15,1156,...|(262145,[15,1156,...|
+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 10 rows

In [12]:
cleaned.select(['label', 'features']).show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(262145,[5213,901...|
 0.0|(262145,[619,1665...|
 0.0|(262145,[1156,384...|
 0.0|(262145,[2265,271...|
 0.0|(262145,[1232,595...|
 0.0|(262145,[619,1156...|
 0.0|(262145,[6962,157...|
 0.0|(262145,[1865,267...|
 0.0|(262145,[24113,51...|
 0.0|(262145,[15,1156,...|
 0.0|(262145,[262,619,...|
 0.0|(262145,[3727,420...|
 0.0|(262145,[966,3535...|
 0.0|(262145,[2306,243...|
 0.0|(262145,[6053,669...|
 0.0|(262145,[16426,20...|
 0.0|(262145,[2326,546...|
 0.0|(262145,[2437,355...|
 0.0|(262145,[14575,14...|
 0.0|(262145,[14,2226,...|
+-----+--------------------+
only showing top 20 rows

In [13]:
from pyspark.ml.classification import LogisticRegression
training, testing = cleaned.randomSplit([0.7, 0.3])
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(training)

In [14]:
test_results = lrModel.transform(testing)
test_results.show(5)


+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 description|fraudulent|length|label| token_text| stop_tokens| hash_token| idf_token| features| rawPrediction| probability|prediction|
+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 We're looking fo...| 0| 457| 0.0|[, we're, looking...|[, looking, motiv...|(262144,[1998,156...|(262144,[1998,156...|(262145,[1998,156...|[6.79154781386242...|[0.99887803134881...| 0.0|
 and that Team bu...| 0| 39| 0.0|[, and, that, tea...|[, team, builds, ...|(262144,[84123,23...|(262144,[84123,23...|(262145,[84123,23...|[2.91829328863521...|[0.94874336628457...| 0.0|
 in a range of ab...| 0| 91| 0.0|[, in, a, range, ...|[, range, 2, kms....|(262144,[97183,10...|(262144,[97183,10...|(262145,[97183,10...|[5.31741508239000...|[0.99511852820612...| 0.0|
 in a range of ab...| 0| 91| 0.0|[, in, a, range, ...|[, range, 2, kms....|(262144,[97183,10...|(262144,[97183,10...|(262145,[97183,10...|[5.31741508239000...|[0.99511852820612...| 0.0|
 in a range of ab...| 0| 91| 0.0|[, in, a, range, ...|[, range, 2, kms....|(262144,[97183,10...|(262144,[97183,10...|(262145,[97183,10...|[5.31741508239000...|[0.99511852820612...| 0.0|
+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Accuracy of model at predicting reviews was:', evaluator.evaluate(test_results))

Accuracy of model at predicting reviews was: 0.8936047774410298

In [16]:
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)
# Tranform the model with the testing data
nbtest_results = predictor.transform(testing)
nbtest_results.show(5)

+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 description|fraudulent|length|label| token_text| stop_tokens| hash_token| idf_token| features| rawPrediction| probability|prediction|
+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 We're looking fo...| 0| 457| 0.0|[, we're, looking...|[, looking, motiv...|(262144,[1998,156...|(262144,[1998,156...|(262145,[1998,156...|[-2491.9153772068...|[1.0,6.2966172133...| 0.0|
 and that Team bu...| 0| 39| 0.0|[, and, that, tea...|[, team, builds, ...|(262144,[84123,23...|(262144,[84123,23...|(262145,[84123,23...|[-111.69322342731...|[0.99999999483343...| 0.0|
 in a range of ab...| 0| 91| 0.0|[, in, a, range, ...|[, range, 2, kms....|(262144,[97183,10...|(262144,[97183,10...|(262145,[97183,10...|[-448.85061874029...|[1.0,6.5144708391...| 0.0|
 in a range of ab...| 0| 91| 0.0|[, in, a, range, ...|[, range, 2, kms....|(262144,[97183,10...|(262144,[97183,10...|(262145,[97183,10...|[-448.85061874029...|[1.0,6.5144708391...| 0.0|
 in a range of ab...| 0| 91| 0.0|[, in, a, range, ...|[, range, 2, kms....|(262144,[97183,10...|(262144,[97183,10...|(262145,[97183,10...|[-448.85061874029...|[1.0,6.5144708391...| 0.0|
+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [17]:
# Use the Class Evaluator for a cleaner description
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(nbtest_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.954509